# Inhibitor Diffusion and Germination - Slow Release

## Numerical experiments - Week 45/2024

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, cuda, float32

## 1. General Assumptions

Medium and conidium sizes:
- The diameter of a spore is assumed to be $5 \mu m$.
- The experiments are performed on a three-dimensional cube lattice, in which a spore is approximated by a sphere of diameter $5 \mu m$ in the middle of the lattice.
- The spherical spore therefore contains ????? lattice sites and has a volume of ??????????????